In [19]:
import tweepy
import json
import pandas as pd
import csv

In [5]:
secrets_file =  'secrets.json'
with open(secrets_file) as f:
    consumer_info = json.load(f)

consumer_key = consumer_info["api_key"]
consumer_secret = consumer_info["api_secret_key"]
access_token = consumer_info["access_token"]
access_token_secret = consumer_info["access_token_secret"]

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [79]:
tweet_ids = pd.read_csv('~/Downloads/election-filter-1.txt', sep = '\n', nrows = 10000, header=None, names=['id'])

In [80]:
tweet_ids.shape

(10000, 1)

In [86]:
tweets = api.statuses_lookup(list(tweet_ids['id'][:100]), include_entities=True, trim_user=True, tweet_mode='extended')



In [70]:
json_str = json.dumps(tweets[4]._json)

In [71]:
json_str

'{"created_at": "Wed Oct 31 12:35:08 +0000 2018", "id": 1057611930024730624, "id_str": "1057611930024730624", "full_text": "RT @JohnBowPapa: Fear, disinformation, propaganda, and prejudice are powerful motivators.  That\'s why despots have employed them throughout\\u2026", "truncated": false, "display_text_range": [0, 140], "entities": {"hashtags": [], "symbols": [], "user_mentions": [{"screen_name": "JohnBowPapa", "name": "John Bowers", "id": 3918372077, "id_str": "3918372077", "indices": [3, 15]}], "urls": []}, "source": "<a href=\\"http://twitter.com/download/iphone\\" rel=\\"nofollow\\">Twitter for iPhone</a>", "in_reply_to_status_id": null, "in_reply_to_status_id_str": null, "in_reply_to_user_id": null, "in_reply_to_user_id_str": null, "in_reply_to_screen_name": null, "user": {"id": 788711426407927808, "id_str": "788711426407927808"}, "geo": null, "coordinates": null, "place": null, "contributors": null, "retweeted_status": {"created_at": "Tue Oct 30 01:41:51 +0000 2018", "id": 10

In [72]:
with open('data/tweets.csv','w') as f:
    writer = csv.writer(f, delimiter= ',')
    writer.writerow(['id', 'created_at', 'text', 'user_id', 'retweets', 'favorites'])
    for status in tweets:
        if not hasattr(status, 'retweeted_status'):
            row = [status.id_str, status.created_at, status.full_text, status.user.id_str, 
                   status.retweet_count, status.favorite_count]
            writer.writerow(row)

In [87]:
with open('data/tweets.csv','w') as f:
    writer = csv.writer(f, delimiter= ',')
    writer.writerow(['id', 'created_at', 'text', 'user_id', 'retweets', 'favorites'])
    
    reader = pd.read_csv('~/Downloads/election-filter-1.txt', sep = '\n', header=None, names=['id'], chunksize = 100)
    for chunk in reader:
        try:
            tweets = api.statuses_lookup(list(chunk['id']), include_entities=True, trim_user=True, tweet_mode='extended')
        except tweepy.RateLimitError:
            time.sleep(15 * 60)
            tweets = api.statuses_lookup(list(chunk['id']), include_entities=True, trim_user=True, tweet_mode='extended')
        for status in tweets:
            if not hasattr(status, 'retweeted_status'):
                row = [status.id_str, status.created_at, status.full_text, status.user.id_str, 
                   status.retweet_count, status.favorite_count]
                writer.writerow(row)
   

KeyboardInterrupt: 

In [73]:
tweet_df = pd.read_csv('data/tweets.csv')

In [74]:
tweet_df.shape

(24, 6)

In [58]:
tweet_df['text'][4]

"RT @JohnBowPapa: Fear, disinformation, propaganda, and prejudice are powerful motivators.  That's why despots have employed them throughout…"